In [1]:
import os
import random
from time import time
import pandas as pd
import numpy as np
import networkx as nx

from utils.utils import *
from utils.graph_creation import *
from utils.motif_counts import *
from utils.visualization import *
import graph_tool.all as gt

In [2]:
sns.set_context("talk")

In [4]:
ver = 630
fdir = "/mnt/cup/labs/seung/research/runzhey/datasets/FlyWire/{}/syn_proof_analysis_ntavg_{}.feather".format(ver, ver)
# fdir = "~/seungmount/research/runzhey/datasets/FlyWire/{}/syn_proof_analysis_ntavg_{}.feather".format(ver, ver)
syn_table = pd.read_feather(fdir)

### 1. Graph Construction

#### Fly

In [5]:
# build index dictionary
cellids =  np.unique(syn_table[["pre_pt_root_id", "post_pt_root_id"]])
nid2cid = {i: cid for i, cid in enumerate(cellids)}
cid2nid = {cid: i for i, cid in enumerate(cellids)}

In [6]:
syn_table["pre_nid"] = pd.Series([cid2nid[cid] for cid in syn_table["pre_pt_root_id"]], 
                                 index=syn_table.index)
syn_table["post_nid"] = pd.Series([cid2nid[cid] for cid in syn_table["post_pt_root_id"]], 
                                 index=syn_table.index)

In [7]:
merged_syn_table = syn_table[["pre_nid", "post_nid", "syn_count"]
                           ].groupby(by=["pre_nid", "post_nid"]).sum().reset_index()

In [8]:
merged_edge_list = [(e[0], e[1], e[2]) for e in merged_syn_table.values]

In [9]:
g_fly = gt.Graph()
g_fly.add_vertex(len(cellids))
e_syn_count = g_fly.new_ep("int32_t")
g_fly.add_edge_list(merged_edge_list, eprops=[e_syn_count])
g_fly.vp["cellid"] = g_fly.new_vp("int64_t")
g_fly.vp["cellid"].a = cellids
g_fly.ep["syn_count"] = e_syn_count

In [10]:
# merged weightedd graph
g_fly

<Graph object, directed, with 127319 vertices and 14680950 edges, 1 internal vertex property, 1 internal edge property, at 0x7fb1b466ff50>

In [11]:
strong_filter = g_fly.new_ep('bool')
for e in g_fly.edges():
    strong_filter[e] = True if g_fly.ep["syn_count"][e] >=5 else False
g_fly.set_edge_filter(strong_filter)
g_fly_th5 = gt.Graph(g_fly, prune=True)

In [12]:
# merged weightedd graph
g_fly_th5

<Graph object, directed, with 127319 vertices and 2613129 edges, 1 internal vertex property, 1 internal edge property, at 0x7fb16ff58190>

#### Worm-male

In [14]:
# SI2 is weighted by synapse count
connmat = pd.read_excel("../ZfishModularity/data/SI2-male-chem.xlsx", engine="openpyxl")

In [15]:
cell_rid2name = dict(connmat["Unnamed: 0"])
cell_name2rid = dict(zip(list(cell_rid2name.values()), range(len(cell_rid2name))))

In [16]:
col_spec_cell_names = list(connmat.keys())[1:]
for n in np.array(connmat["Unnamed: 0"]):
    col_spec_cell_names.remove(n)

In [17]:
# rearrange column order
cell_names = list(connmat["Unnamed: 0"]) + col_spec_cell_names

In [18]:
connmat = connmat[cell_names]

In [19]:
g_worm_male = gt.Graph()
idx = np.array(connmat.fillna(0)).nonzero()
g_worm_male.add_edge_list(np.transpose(idx))

ew = g_worm_male.new_edge_property("int32_t")
ew.a = np.array(connmat.fillna(0))[idx] 
g_worm_male.ep['syn_count'] = ew

vw = g_worm_male.new_vertex_property("string")
for i, v in enumerate(g_worm_male.vertices()):
    vw[v] = cell_names[i]
g_worm_male.vp['cell_name'] = vw

In [20]:
cell_type = pd.read_excel("../ZfishModularity/data/SI4-combined-celltype.xlsx", engine="openpyxl")

In [21]:
cell_name2type = dict(zip(list(cell_type["cell name"]), list(cell_type["cell category"])))

In [22]:
vw = g_worm_male.new_vertex_property("string")
for i, v in enumerate(g_worm_male.vertices()):
    name = g_worm_male.vp['cell_name'][v]
    if name in cell_name2type.keys():
        vw[v] = cell_name2type[name]
    else:
        vw[v] = "others"
g_worm_male.vp['cell_type'] = vw

In [23]:
neuron_filter = g_worm_male.new_vertex_property("bool")
for v in g_worm_male.vertices():
    neuron_filter[v] = g_worm_male.vp['cell_type'][v] not in ['endorgan', 'muscle']

In [24]:
g_worm_male.set_vertex_filter(neuron_filter)
g_worm_male = gt.Graph(g_worm_male, prune=True)

In [25]:
g_worm_male

<Graph object, directed, with 364 vertices and 3467 edges, 2 internal vertex properties, 1 internal edge property, at 0x7faeea2d4e10>

#### Worm-herm

In [26]:
# SI2 is weighted by synapse count
connmat = pd.read_excel("../ZfishModularity/data/SI2-herm-chem.xlsx", engine="openpyxl")

In [27]:
cell_rid2name = dict(connmat["Unnamed: 0"])
cell_name2rid = dict(zip(list(cell_rid2name.values()), range(len(cell_rid2name))))

In [28]:
col_spec_cell_names = list(connmat.keys())[1:]
for n in np.array(connmat["Unnamed: 0"]):
    col_spec_cell_names.remove(n)

In [29]:
# rearrange column order
cell_names = list(connmat["Unnamed: 0"]) + col_spec_cell_names

In [30]:
connmat = connmat[cell_names]

In [31]:
g_worm_herm = gt.Graph()
idx = np.array(connmat.fillna(0)).nonzero()
g_worm_herm.add_edge_list(np.transpose(idx))

ew = g_worm_herm.new_edge_property("int32_t")
ew.a = np.array(connmat.fillna(0))[idx] 
g_worm_herm.ep['syn_count'] = ew

vw = g_worm_herm.new_vertex_property("string")
for i, v in enumerate(g_worm_herm.vertices()):
    vw[v] = cell_names[i]
g_worm_herm.vp['cell_name'] = vw

In [32]:
cell_type = pd.read_excel("../ZfishModularity/data/SI4-combined-celltype.xlsx", engine="openpyxl")

In [33]:
cell_name2type = dict(zip(list(cell_type["cell name"]), list(cell_type["cell category"])))

In [34]:
vw = g_worm_herm.new_vertex_property("string")
for i, v in enumerate(g_worm_herm.vertices()):
    name = g_worm_herm.vp['cell_name'][v]
    if name in cell_name2type.keys():
        vw[v] = cell_name2type[name]
    else:
        vw[v] = "others"
g_worm_herm.vp['cell_type'] = vw

In [35]:
neuron_filter = g_worm_herm.new_vertex_property("bool")
for v in g_worm_herm.vertices():
    neuron_filter[v] = g_worm_herm.vp['cell_type'][v] not in ['endorgan', 'muscle']

In [36]:
g_worm_herm.set_vertex_filter(neuron_filter)
g_worm_herm = gt.Graph(g_worm_herm, prune=True)

In [37]:
g_worm_herm

<Graph object, directed, with 302 vertices and 3242 edges, 2 internal vertex properties, 1 internal edge property, at 0x7faeea2eded0>

#### Zebrafish center subgraph

In [38]:
g_zfish = gt.load_graph("../ZfishModularity/saved/gt-ax50-dd100.xml.gz")

In [39]:
g_zfish.ep["syn_count"] = g_zfish.ep["#synapses"]

In [40]:
g_zfish

<Graph object, directed, with 419 vertices and 5605 edges, 18 internal vertex properties, 3 internal edge properties, at 0x7faeea2f03d0>

#### Pinky th100

In [41]:
pyc_synapses = pd.read_csv('../PinkyMotif/motif_cleaned/motif_analysis/data/synapses.csv')
pyc_soma = pd.read_csv('../PinkyMotif/motif_cleaned/motif_analysis/data/soma.csv')

In [42]:
def get_thresholded_graph(g, axls, th):
    rg = g.copy()
    for n in g.nodes:
        if axls[n] <= th:
            rg.remove_node(n)
    return rg

def remove_autapses(g):
    rg = g.copy()
    for e in g.edges:
        if e[0] == e[1]:
            rg.remove_edge(*e)
    return rg

def synapses_to_connections(df):
    return df[['pre','post']].drop_duplicates(subset=['pre','post'])

In [43]:
# create the simple directed graph (no self-loops, no multi-edges)
# treat multiple synapses as one connection
pyc_subgraph = synapses_to_connections(pyc_synapses[['pre','post']])
# remove self-loops
g_actual = remove_autapses(edges_to_graph(pyc_subgraph))
g_actual.add_nodes_from(pyc_soma["segment_id"])

In [44]:
print(nx.info(g_actual))

Name: 
Type: DiGraph
Number of nodes: 363
Number of edges: 1750
Average in degree:   4.8209
Average out degree:   4.8209


In [45]:
axls = {n:np.array(pyc_soma[pyc_soma["segment_id"] == n]["axon_len"])[0] for n in g_actual.nodes()}

In [46]:
g_th100 = get_thresholded_graph(g_actual, axls, 100)
print(nx.info(g_th100))

Name: 
Type: DiGraph
Number of nodes: 111
Number of edges: 659
Average in degree:   5.9369
Average out degree:   5.9369


In [47]:
g_mouse = nx2gt(g_th100)

In [48]:
g_mouse.ep["syn_count"] = g_mouse.new_edge_property("int32_t")
for e in g_mouse.edges():
    g_mouse.ep["syn_count"][e] = len(pyc_synapses[(pyc_synapses["pre"] == int(g_mouse.vp["id"][e.source()])) 
                                                  & (pyc_synapses["post"] == int(g_mouse.vp["id"][e.target()]))])

In [49]:
g_mouse

<Graph object, directed, with 111 vertices and 659 edges, 1 internal vertex property, 1 internal edge property, at 0x7faeea2c2910>

### Comparison

In [50]:
g_fly.clear_filters()

In [51]:
g_nns = g_fly, g_fly_th5, g_worm_herm, g_worm_male, g_zfish, g_mouse

In [52]:
g_nns

(<Graph object, directed, with 127319 vertices and 14680950 edges, 1 internal vertex property, 1 internal edge property, at 0x7fb1b466ff50>,
 <Graph object, directed, with 127319 vertices and 2613129 edges, 1 internal vertex property, 1 internal edge property, at 0x7fb16ff58190>,
 <Graph object, directed, with 302 vertices and 3242 edges, 2 internal vertex properties, 1 internal edge property, at 0x7faeea2eded0>,
 <Graph object, directed, with 364 vertices and 3467 edges, 2 internal vertex properties, 1 internal edge property, at 0x7faeea2d4e10>,
 <Graph object, directed, with 419 vertices and 5605 edges, 18 internal vertex properties, 3 internal edge properties, at 0x7faeea2f03d0>,
 <Graph object, directed, with 111 vertices and 659 edges, 1 internal vertex property, 1 internal edge property, at 0x7faeea2c2910>)

In [53]:
conn_p = [g.num_edges() / g.num_vertices() / (g.num_vertices()-1) for g in g_nns]
conn_p

[0.0009056723997342875,
 0.00016120474575863678,
 0.03566478185298454,
 0.02623891260254897,
 0.03200260360164894,
 0.05397215397215398]

In [54]:
np.array(conn_p) / conn_p[1]

array([  5.61814974,   1.        , 221.23903167, 162.76761878,
       198.52147312, 334.80499422])

In [55]:
for g in g_nns:
    print(sum(g.ep["syn_count"].a)/g.num_edges())

3.5947458441040943
12.617289846769907
3.147131400370142
3.594750504759158
1.6897413024085637
1.1411229135053111


In [56]:
for g in g_nns:
    print(max(g.ep["syn_count"].a))

2358
2358
36
63
21
5


In [57]:
reciprocities = [gt.edge_reciprocity(g) for g in g_nns]
reciprocities

[0.26551919324021944,
 0.13837740119221056,
 0.3723010487353485,
 0.3862128641476781,
 0.11311329170383586,
 0.08801213960546282]

In [58]:
np.array(reciprocities) / np.array(conn_p)

array([293.17355074, 858.39533161,  10.43889881,  14.71908802,
         3.53450279,   1.63069533])

In [59]:
%%time
cluster_coefs = [gt.global_clustering(g)[0] for g in g_nns]
cluster_coefs

CPU times: user 2min 6s, sys: 967 ms, total: 2min 7s
Wall time: 9.14 s


[0.10804973088935205,
 0.04632866099163119,
 0.28430426432509387,
 0.33116515048737655,
 0.18198723638054182,
 0.15868044681073182]

In [60]:
%%time
conn_p = np.array(conn_p)
undir_conn_p = 1 - (1-conn_p)**2
cluster_coefs / undir_conn_p

CPU times: user 327 µs, sys: 12 µs, total: 339 µs
Wall time: 355 µs


array([ 59.67868841, 143.70667338,   4.05815096,   6.39446497,
         2.88955588,   1.51079178])

In [61]:
%%time
cfg_rec = []
cfg_cls = []
for i, g in enumerate(g_nns):
    cfg_rec.append([])
    cfg_cls.append([])
    cfg_g = gt.Graph(g)
    for _ in range(100):
        gt.random_rewire(cfg_g)
        cfg_rec[i].append(gt.edge_reciprocity(cfg_g))
        cfg_cls[i].append(gt.global_clustering(cfg_g)[0])

CPU times: user 6h 18min 15s, sys: 2min 3s, total: 6h 20min 18s
Wall time: 1h 19min 53s


In [62]:
reciprocities / np.mean(cfg_rec, axis=1)

array([46.07720019, 43.7444805 ,  4.98554316,  5.95932173,  2.60540807,
        1.41050584])

In [63]:
cluster_coefs / np.mean(cfg_cls, axis=1)

array([9.17835287, 7.56014674, 1.86857217, 2.40084297, 1.8933038 ,
       1.05695749])